In [3]:
from mxnet import nd
from mxnet.gluon import nn

In [5]:
class MLP(nn.Block):   #重载 重载发生在本类,方法名相同,参数列表不同,与返回值无关,只和方法名,参数列表,参数的类型有关.
    # 声明带有模型参数的层，这⾥我们声明了两个全连接层。
    def __init__(self, **kwargs):
    # 调⽤MLP ⽗类Block 的构造函数来进⾏必要的初始化。这样在构造实例时还可以指定
    # 其他函数参数，例如后⾯章节将介绍的模型参数params。
        super(MLP, self).__init__(**kwargs)  #super 找到mlp的父类，将mlp的对象转换成父类的对象
        self.hidden = nn.Dense(256, activation='relu') # 隐藏层。
        self.output = nn.Dense(10) # 输出层。
    # 定义模型的前向计算，即如何根据输⼊x 计算返回所需要的模型输出。
    def forward(self, x):
        return self.output(self.hidden(x))

In [8]:
x = nd.random.uniform(shape = (2,20))  
net = MLP()
net.initialize()
net(x)


[[ 0.02394444  0.05150646  0.01035161 -0.06435342 -0.05801985  0.06419201
   0.04472664 -0.01852541 -0.03237377  0.0738937 ]
 [ 0.05207362  0.04186264  0.04021509 -0.06558425 -0.02249499  0.03413139
   0.02135914 -0.06898529  0.02329671  0.00336681]]
<NDArray 2x10 @cpu(0)>

In [13]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # 使⽤get_constant 创建的随机权重参数不会在训练中被迭代（即常数参数）。
        self.rand_weight = self.params.get_constant('rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')
    def forward(self, x):
        x = self.dense(x)
        # 使⽤创建的常数参数，以及NDArray 的relu 和dot 函数。
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        # 重⽤全连接层。等价于两个全连接层共享参数。
        x = self.dense(x)
        # 控制流，这⾥我们需要调⽤asscalar 来返回标量进⾏⽐较。
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

In [14]:
net = FancyMLP()
net.initialize()
net(x)


[22.833347]
<NDArray 1 @cpu(0)>

In [11]:
#由于FancyMLP 和Sequential 类都是Block 类的⼦类，我们可以嵌套调⽤它们。
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')
    def forward(self, x):
        return self.dense(self.net(x))

In [12]:
net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())
net.initialize()
net(x)


[3.5920622]
<NDArray 1 @cpu(0)>